In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv("./NFWBS_PUF_2016_data.csv")
df.head()

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.327561
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.410871
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.260668


In [15]:
list(df.columns)

['PUF_ID',
 'sample',
 'fpl',
 'SWB_1',
 'SWB_2',
 'SWB_3',
 'FWBscore',
 'FWB1_1',
 'FWB1_2',
 'FWB1_3',
 'FWB1_4',
 'FWB1_5',
 'FWB1_6',
 'FWB2_1',
 'FWB2_2',
 'FWB2_3',
 'FWB2_4',
 'FSscore',
 'FS1_1',
 'FS1_2',
 'FS1_3',
 'FS1_4',
 'FS1_5',
 'FS1_6',
 'FS1_7',
 'FS2_1',
 'FS2_2',
 'FS2_3',
 'SUBKNOWL1',
 'ACT1_1',
 'ACT1_2',
 'FINGOALS',
 'PROPPLAN_1',
 'PROPPLAN_2',
 'PROPPLAN_3',
 'PROPPLAN_4',
 'MANAGE1_1',
 'MANAGE1_2',
 'MANAGE1_3',
 'MANAGE1_4',
 'SAVEHABIT',
 'FRUGALITY',
 'AUTOMATED_1',
 'AUTOMATED_2',
 'ASK1_1',
 'ASK1_2',
 'SUBNUMERACY2',
 'SUBNUMERACY1',
 'CHANGEABLE',
 'GOALCONF',
 'LMscore',
 'FINKNOWL1',
 'FINKNOWL2',
 'FINKNOWL3',
 'FK1correct',
 'FK2correct',
 'FK3correct',
 'KHscore',
 'KHKNOWL1',
 'KHKNOWL2',
 'KHKNOWL3',
 'KHKNOWL4',
 'KHKNOWL5',
 'KHKNOWL6',
 'KHKNOWL7',
 'KHKNOWL8',
 'KHKNOWL9',
 'KH1correct',
 'KH2correct',
 'KH3correct',
 'KH4correct',
 'KH5correct',
 'KH6correct',
 'KH7correct',
 'KH8correct',
 'KH9correct',
 'ENDSMEET',
 'HOUSING',
 'LIVING

In [16]:
list(df.isnull().any())

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,


We can see that none of the columns contain any NULL values, meaning that we are **not** missing any data. However, it is important to note that our dataset is a survey, so most of our columns are survey questions and some of the allowed responses include "Refused" (represented by the value -1). We also have "Response not written to dataset" (represented by the value -4). In order to get the response rate for the survey questions, we will create a copy of the dataframe and replace these values with None/Null, given that they don't answer the survey question asked. Keep in mind this is just a check; we are not getting rid of these values at this point.

In [21]:
nonresponse_values = [-1, -4]
temp_df = df.copy()
cols = temp_df.columns
for col in cols:
    col_values = temp_df[col]
    temp_df[col] = np.where(col_values.isin(nonresponse_values),None,temp_df[col])
temp_df

,PUF_ID,sample,fpl,SWB_1,SWB_2,SWB_3,FWBscore,FWB1_1,FWB1_2,FWB1_3,...,PPMSACAT,PPREG4,PPREG9,PPT01,PPT25,PPT612,PPT1317,PPT18OV,PCTLT200FPL,finalwt
0,10350,2,3,5,5,6,55,3,3,3,...,1,4,8,0,0,0,0,1,0,0.367292
1,7740,1,3,6,6,6,51,2,2,3,...,1,2,3,0,0,0,0,2,0,1.32756
2,13699,1,3,4,3,4,49,3,3,3,...,1,4,9,0,0,0,1,2,1,0.835156
3,7267,1,3,6,6,6,49,3,3,3,...,1,3,7,0,0,0,0,1,0,1.41087
4,7375,1,3,4,4,4,49,3,3,3,...,1,2,4,0,0,1,0,4,1,4.26067
5,10910,1,3,5,7,5,67,5,1,1,...,1,2,3,0,0,0,0,2,0,0.760061
6,11079,1,3,5,4,7,51,1,3,2,...,1,2,4,0,0,1,1,3,0,1.37052
7,7741,1,3,2,2,2,47,1,2,4,...,1,2,4,0,0,0,0,3,0,1.19337
8,8303,1,3,7,7,7,43,4,5,2,...,1,3,7,0,0,0,1,3,0,2.49784
9,9688,1,3,4,5,4,58,3,3,2,...,1,1,2,0,0,0,0,3,0,1.2666
